In [1]:
import pandas as pd

import sys
sys.path.append('..')
from Analysis.config import Config


In [2]:
# Load sqlite3
import sqlite3
conn = sqlite3.connect("../rosteriq.db")
cursor = conn.cursor()

In [12]:
# Add new column
cursor.execute("ALTER TABLE Player_Seasons ADD COLUMN player_cluster INT")
conn.commit()

In [3]:
# Add the player cluster numbers to the database
position = ["G", "F", "C"]
for year in range(Config.START_YEAR, Config.END_YEAR_EXCLUDE):
    for pos in position:
        df = pd.read_csv(f"../Analysis/Clustering/Players/{year}/KClustering/player_labels_{pos}.csv")

        for _, plyr in df.iterrows():
            id = plyr["player_id"]
            team_name = plyr['team_name']
            cluster = plyr["Cluster"]
            season_year = plyr['season_year']
            cursor.execute("""
                UPDATE Player_Seasons 
                SET player_cluster = ? 
                WHERE player_id = ? 
                AND team_name = ? 
                AND season_year = ? 
                AND player_cluster IS NULL
            """, (cluster, id, team_name, season_year))
    
    conn.commit()  # Commit once per year instead of per position

KeyboardInterrupt: 

In [15]:
# Rename team cluster name
cursor.execute("""ALTER TABLE Team_Seasons
RENAME COLUMN cluster TO team_cluster;""")

In [4]:
# Add the team cluster numbers to the database
for year in range(Config.START_YEAR, Config.END_YEAR_EXCLUDE):
    team_df = pd.read_csv(f"../Analysis/Clustering/Teams/{year}/KClustering/labels.csv")

    for _, team in team_df.iterrows():
        team_name = team['team_name']
        season_year = team['season_year']
        team_cluster = team['team_cluster']
        cursor.execute("""
                         UPDATE Team_Seasons
                         SET team_cluster = ?
                         WHERE team_name = ? AND season_year = ?""", 
                         (team_cluster, team_name, season_year))
    
    conn.commit()

conn.close()

In [3]:
# Delete an old table
cursor.execute("DROP TABLE TempMatches;")